In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import pandas as pd
import time

In [2]:
# 웹드라이버 실행
driver = webdriver.Chrome()
driver.maximize_window()

In [3]:
# 사이트를 요청한다.
driver.get('https://m.sports.naver.com/kbaseball/schedule/index?category=kbo&date=2008-03-29')

In [4]:
# 한 페이지의 데이터를 가져오는 함수
def get_page_data(page_data, is_post_season) :

    time.sleep(1)

    # 수집한 데이터를 담을 딕셔너리
    d1 = {
        '경기날짜' : [],
        '팀1이름': [],
        '팀2이름': [],
        '팀1점수' : [],
        '팀2점수' : []
    }

    # 현재 연도를 가져온다.
    year_tag = page_data.find_element(by=By.CSS_SELECTOR, value='#content > div > div:nth-child(3) > div > div.CalendarDate_calendar_date_group__bXTAD > div > div.CalendarDate_current_date_wrap__BSh44 > time > span')
                                                                 

    # print(year_tag.text)

    # 경기 결과 목록들을 가지고 있는 root를 가져온다.
    # 만약 포스트 시즌이라면
    if is_post_season == True :
        result_root = page_data.find_element(by=By.XPATH, value='//*[@id="content"]/div/div[4]/div[2]')
    # 포스트 시즌이 아니라면
    else :
        result_root = page_data.find_element(by=By.CSS_SELECTOR, value='#content > div > div:nth-child(5) > div')
    # 바로 하위에 있는 div 태그들을 가져온다.
    result_list = result_root.find_elements(by=By.XPATH, value='./child::div')
    # print(len(result_list))

    # 가져온 경기 결과 수 만큼 반복한다.
    for result_tag in result_list :
        # 현재 경기의 날짜를 가져온다.
        date_tag = result_tag.find_element(by=By.XPATH, value='./child::div/div/em')
        # print(date_tag.text)
        # 띄어 쓰기를 기준으로 잘라낸다.
        date_split = date_tag.text.split()
        # 날짜 문자열을 만들어준다.
        date_str = f'{year_tag.text}-{date_split[0][:-1]}-{date_split[1][:-1]}'
        # print(date_str)

        # 각 날짜별 경기 결과가 담겨져 있는 ul 태그를 가져온다.
        ul_tag = result_tag.find_element(by=By.XPATH, value='./child::ul')
        # print(ul_tag)
        # ul_tag 안에 있는 li 태그 들을 가져온다.
        li_list = ul_tag.find_elements(by=By.XPATH, value='./child::li')
        # print(len(li_list))
        # li 태그의 수 만큼 반복한다.
        for li_tag in li_list :
            # 경기 결과가 담겨져 있는 div 태그 부분을 가져온다.
            date_result_div = li_tag.find_element(by=By.XPATH, value='./child::div/div[2]')
            # print(date_result_div.text)
            # 경기 결과를 나타내는 문자열 값을 가져온다.
            result_str_tag = date_result_div.find_element(by=By.XPATH, value='./child::em')
            # print(result_str_tag.text)
            # 경기 결과를 나타내는 문자열이 '종료' 인  경우에만 수집을 한다.
            if result_str_tag.text.strip() == '종료' :
                # 좌측 팀의 정보가 담긴 div 태그를 가져온다.
                left_team_div = date_result_div.find_element(by=By.XPATH, value='./child::div[1]')
                # print(left_team_div.text)
                # 우측 팀의 정보가 담기 div 태그를 가져온다.
                right_team_div = date_result_div.find_element(by=By.XPATH, value='./child::div[2]')
                # print(right_team_div.text)

                # 좌측팀의 이름을 가져온다.
                left_team_name_tag = left_team_div.find_element(by=By.XPATH, value='./child::div/div[2]/div/strong')
                # print(left_team_name_tag.text)

                # 우측팀의 이름을 가져온다.
                right_team_name_tag = right_team_div.find_element(by=By.XPATH, value='./child::div/div[2]/div/strong')
                # print(right_team_name_tag.text)

                # 좌측팀의 점수를 가져온다.
                left_team_point_tag = left_team_div.find_element(by=By.XPATH, value='./child::div[2]/strong')
                # print(left_team_point_tag.text)

                # 우측팀의 점수를 가져온다.
                right_team_point_tag = right_team_div.find_element(by=By.XPATH, value='./child::div[2]/strong')
                # print(right_team_point_tag.text)

                # 데이터를 담는다.
                d1['경기날짜'].append(date_str)
                d1['팀1이름'].append(left_team_name_tag.text.strip())
                d1['팀2이름'].append(right_team_name_tag.text.strip())
                d1['팀1점수'].append(left_team_point_tag.text.strip())
                d1['팀2점수'].append(right_team_point_tag.text.strip())

    df1 = pd.DataFrame(d1)
    # display(df1)
    return df1

In [5]:
# 현재 년도의 월 데이터를 가져온다.
def get_month_date(page_data) :
    time.sleep(1)
    # 월과 포스트시즌 등이 있는 부분을 가져온다.
    month_div = page_data.find_element(by=By.XPATH, value='//*[@id="content"]/div/div[3]/div/div[2]/div/div/div')
    # print(month_div.text)

    # 내부의 div 태그들을 가져온다.
    month_list = month_div.find_elements(by=By.XPATH, value='./child::div')
    for month_tag in month_list :
        time.sleep(1)
        # print(month_tag.text)
        # 내부의 button를 가져와 눌러준다.
        month_button = month_tag.find_element(by=By.XPATH, value='./child::button')
        month_button = month_tag.find_element(by=By.XPATH, value='./child::button')

        driver.execute_script("arguments[0].scrollIntoView(true);", month_button)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", month_button)

        time.sleep(1)


        # 현재가 포스트 시즌이라면..
        if month_button.text.strip() == "포스트시즌" :
            # 포스트시즌 탭 목록을 가져온다
            post_tab_tag = page_data.find_element(by=By.XPATH, value='//*[@id="content"]/div/div[4]/div[1]/div/div')
            # print(post_tab_tag)

            # 이 탭은 탭 버튼을 누를 때마다 탭 버튼을 새롭게 만들고 있다.
            # 이에 탭 누르기 전에 다시 가져와서 누르는 것으로 처리를 하겠다.
            # 내부의 div 태그들을 가져온다.(전체가 몇개인지 확인하기 위해서....)
            post_tab_div_list = post_tab_tag.find_elements(by=By.XPATH, value='./child::div')
            tab_count = len(post_tab_div_list)
            # 가져온 탭의 개수만큼 반복한다.
            for position in range(tab_count) :
                time.sleep(1)
                # 포스트시즌 탭 목록을 가져온다
                post_tab_tag = page_data.find_element(by=By.XPATH, value='//*[@id="content"]/div/div[4]/div[1]/div/div')
                # 탭 버튼 div 태그들을 다시 가져온다.
                post_tab_div_list = post_tab_tag.find_elements(by=By.XPATH, value='./child::div')
                # 현재 번째의 div 태그에서 버튼 태그를 가져온다.
                post_tab_button_tag = post_tab_div_list[position].find_element(by=By.XPATH, value='./child::button')
                # 누른다.
                post_tab_button_tag.click()

                # 데이터를 수집한다.
                df1 = get_page_data(page_data, True)
                display(df1)
        else :
            df1 = get_page_data(page_data, False)
            display(df1)

        
        

get_month_date(driver)

,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-3-29,KIA,삼성,3,4
1,2008-3-29,LG,SK,4,5
2,2008-3-29,롯데,한화,11,1
3,2008-3-30,롯데,한화,9,8
4,2008-3-30,KIA,삼성,0,3
5,2008-3-30,넥센,두산,1,4
6,2008-3-30,LG,SK,3,1


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-4-1,한화,넥센,5,6
1,2008-4-1,두산,KIA,3,0
2,2008-4-1,SK,롯데,4,8
3,2008-4-1,삼성,LG,6,2
4,2008-4-2,한화,넥센,4,5
...,...,...,...,...,...
94,2008-4-29,넥센,삼성,5,6
95,2008-4-30,KIA,두산,6,7
96,2008-4-30,LG,롯데,4,3
97,2008-4-30,SK,한화,2,6


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-5-1,KIA,두산,1,5
1,2008-5-1,LG,롯데,5,8
2,2008-5-1,SK,한화,6,4
3,2008-5-1,넥센,삼성,2,0
4,2008-5-3,두산,LG,16,4
...,...,...,...,...,...
96,2008-5-30,SK,삼성,6,7
97,2008-5-31,KIA,두산,8,3
98,2008-5-31,LG,한화,11,7
99,2008-5-31,롯데,넥센,11,2


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-6-1,LG,한화,8,2
1,2008-6-1,KIA,두산,6,2
2,2008-6-1,롯데,넥센,8,3
3,2008-6-1,SK,삼성,18,0
4,2008-6-3,한화,KIA,4,1
...,...,...,...,...,...
84,2008-6-28,삼성,두산,7,0
85,2008-6-29,한화,SK,1,2
86,2008-6-29,KIA,롯데,4,5
87,2008-6-29,LG,넥센,1,4


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-7-1,롯데,삼성,3,7
1,2008-7-1,두산,한화,4,6
2,2008-7-1,SK,LG,2,4
3,2008-7-2,두산,한화,8,6
4,2008-7-2,넥센,KIA,4,3
...,...,...,...,...,...
82,2008-7-30,롯데,두산,9,3
83,2008-7-31,한화,넥센,4,2
84,2008-7-31,LG,KIA,0,5
85,2008-7-31,롯데,두산,9,6


...

,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-10-8,삼성,롯데,12,3
1,2008-10-9,삼성,롯데,4,3
2,2008-10-11,롯데,삼성,0,0


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-10-16,삼성,두산,4,8
1,2008-10-17,삼성,두산,7,4
2,2008-10-19,두산,삼성,2,6
3,2008-10-20,두산,삼성,12,6
4,2008-10-21,두산,삼성,6,4
5,2008-10-23,삼성,두산,2,5


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-10-26,두산,SK,5,2
1,2008-10-27,두산,SK,2,5
2,2008-10-29,SK,두산,3,2
3,2008-10-30,SK,두산,4,1
4,2008-10-31,SK,두산,2,0


In [6]:
# 다음 년도로 이동할 버튼을 누르는 함수
# False를 반환하면 더이상 이동할 곳이 없다고 한다.
def next_button_click(page_data) :
    next_button_tag = page_data.find_element(by=By.XPATH, value='//*[@id="content"]/div/div[3]/div/div[1]/div/div[1]/button[2]')
    # 버튼이 활성화 되어 있으면 누르고 True를 반환한다.
    if next_button_tag.get_attribute('disabled') == None :
        next_button_tag.click()
        return True
    else :
        return False

In [7]:
# 수집한다.
while True :
    # 각 월을 다니면서 데이터를 수집한다.
    df1 = get_month_date(driver)
    display(df1)
    # 다음 년도로 이동한다.
    chk1 = next_button_click(driver)
    # 만약 이동에 실패했다면 while문을 중단한다.
    if chk1 == False :
        break

,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-3-29,KIA,삼성,3,4
1,2008-3-29,LG,SK,4,5
2,2008-3-29,롯데,한화,11,1
3,2008-3-30,롯데,한화,9,8
4,2008-3-30,KIA,삼성,0,3
5,2008-3-30,넥센,두산,1,4
6,2008-3-30,LG,SK,3,1


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-4-1,한화,넥센,5,6
1,2008-4-1,두산,KIA,3,0
2,2008-4-1,SK,롯데,4,8
3,2008-4-1,삼성,LG,6,2
4,2008-4-2,한화,넥센,4,5
...,...,...,...,...,...
94,2008-4-29,넥센,삼성,5,6
95,2008-4-30,KIA,두산,6,7
96,2008-4-30,LG,롯데,4,3
97,2008-4-30,SK,한화,2,6


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-5-1,KIA,두산,1,5
1,2008-5-1,LG,롯데,5,8
2,2008-5-1,SK,한화,6,4
3,2008-5-1,넥센,삼성,2,0
4,2008-5-3,두산,LG,16,4
...,...,...,...,...,...
96,2008-5-30,SK,삼성,6,7
97,2008-5-31,KIA,두산,8,3
98,2008-5-31,LG,한화,11,7
99,2008-5-31,롯데,넥센,11,2


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-6-1,LG,한화,8,2
1,2008-6-1,KIA,두산,6,2
2,2008-6-1,롯데,넥센,8,3
3,2008-6-1,SK,삼성,18,0
4,2008-6-3,한화,KIA,4,1
...,...,...,...,...,...
84,2008-6-28,삼성,두산,7,0
85,2008-6-29,한화,SK,1,2
86,2008-6-29,KIA,롯데,4,5
87,2008-6-29,LG,넥센,1,4


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2008-7-1,롯데,삼성,3,7
1,2008-7-1,두산,한화,4,6
2,2008-7-1,SK,LG,2,4
3,2008-7-2,두산,한화,8,6
4,2008-7-2,넥센,KIA,4,3
...,...,...,...,...,...
82,2008-7-30,롯데,두산,9,3
83,2008-7-31,한화,넥센,4,2
84,2008-7-31,LG,KIA,0,5
85,2008-7-31,롯데,두산,9,6


...

,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2023-11-2,KT,NC,3,0
1,2023-11-3,KT,NC,11,2
2,2023-11-5,NC,KT,2,3
3,2023-11-7,KT,LG,3,2
4,2023-11-8,KT,LG,4,5
5,2023-11-10,LG,KT,8,7
6,2023-11-11,LG,KT,15,4
7,2023-11-13,KT,LG,2,6


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2023-11-2,KT,NC,3,0
1,2023-11-3,KT,NC,11,2
2,2023-11-5,NC,KT,2,3
3,2023-11-7,KT,LG,3,2
4,2023-11-8,KT,LG,4,5
5,2023-11-10,LG,KT,8,7
6,2023-11-11,LG,KT,15,4
7,2023-11-13,KT,LG,2,6


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2023-11-2,KT,NC,3,0
1,2023-11-3,KT,NC,11,2
2,2023-11-5,NC,KT,2,3
3,2023-11-7,KT,LG,3,2
4,2023-11-8,KT,LG,4,5
5,2023-11-10,LG,KT,8,7
6,2023-11-11,LG,KT,15,4
7,2023-11-13,KT,LG,2,6


,경기날짜,팀1이름,팀2이름,팀1점수,팀2점수
0,2023-11-2,KT,NC,3,0
1,2023-11-3,KT,NC,11,2
2,2023-11-5,NC,KT,2,3
3,2023-11-7,KT,LG,3,2
4,2023-11-8,KT,LG,4,5
5,2023-11-10,LG,KT,8,7
6,2023-11-11,LG,KT,15,4
7,2023-11-13,KT,LG,2,6


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div/div[4]/div[1]/div/div"}
  (Session info: chrome=143.0.7499.170); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
0   chromedriver                        0x0000000104c08b2c cxxbridge1$str$ptr + 3033596
1   chromedriver                        0x0000000104c00b30 cxxbridge1$str$ptr + 3000832
2   chromedriver                        0x00000001046fab0c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74196
3   chromedriver                        0x00000001047419c8 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 364688
4   chromedriver                        0x0000000104782968 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 630832
5   chromedriver                        0x000000010473616c _RNvCsgXDX2mvAJA